In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
%cd /content/drive/MyDrive/Foodalyze

/content/drive/MyDrive/Foodalyze


In [13]:
!pip install dvc[all] ultralytics mlflow

In [14]:
!git clone https://github.com/alina1114/Foodalyze.git
%cd Foodalyze

Cloning into 'Foodalyze'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 23 (delta 2), reused 18 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 6.47 KiB | 828.00 KiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/drive/MyDrive/Foodalyze/Foodalyze


In [15]:
DATA_PATH = "/content/drive/MyDrive/Foodalyze/data/IndianFoodDatasetFinalFiltered"


In [46]:
!pip uninstall -y wandb
!pip install --upgrade ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.9 MB/s eta 0:00:00
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.0
    Uninstalling ultralytics-8.3.0:
      Successfully uninstalled ultralytics-8.3.0


In [4]:
import os

# Root directory of dataset
root = "/content/drive/MyDrive/Foodalyze/data/IndianFoodDatasetFinalFiltered"

# Define your class names
classes = [
    "aloo_gobi","aloo_matar","aloo_methi","aloo_shimla_mirch","aloo_tikki","bhatura",
    "bhindi_masala",
    "biryani",
    "boondi",
    "butter_chicken",
    "chak_hao_kheer",
    "cham_cham",
    "chana_masala",
    "chapati",
    "chhena_kheeri",
    "chicken_razala",
    "chicken_tikka",
    "chicken_tikka_masala",
    "chikki",
    "daal_puri",
    "dal_makhani",
    "dal_tadka",
    "dharwad_pedha",
    "double_ka_meetha",
    "dum_aloo",
    "gajar_ka_halwa",
    "gulab_jamun",
    "imarti",
    "jalebi",
    "kachori",
    "kadai_paneer",
    "kajjikaya",
    "kalakand",
    "karela_bharta",
    "kofta",
    "lassi",
    "makki_di_roti_sarson_da_saag",
    "malapua",
    "misi_roti",
    "misti_doi",
    "mysore_pak",
    "naan",
    "navrattan_korma",
    "palak_paneer",
    "paneer_butter_masala",
    "poha",
    "poornalu",
    "pootharekulu",
    "qubani_ka_meetha",
    "rabri",
    "ras_malai",
    "rasgulla",
    "sandesh",
    "sheer_korma",
    "sheera",
    "sohan_halwa",
    "sohan_papdi",
    "unni_appam"
]

# Create mapping: name -> index
class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}

# Go through all label files
for split in ["train", "valid"]:
    labels_dir = os.path.join(root, split, "labels")
    for file in os.listdir(labels_dir):
        path = os.path.join(labels_dir, file)
        lines = open(path).read().strip().splitlines()
        new_lines = []
        for line in lines:
            parts = line.split()
            if parts[0] in class_to_idx:
                cls_id = class_to_idx[parts[0]]
                new_line = " ".join([str(cls_id)] + parts[1:])
                new_lines.append(new_line)
            else:
                print(f"Unknown label name: {parts[0]} in {file}")
        with open(path, "w") as f:
            f.write("\n".join(new_lines))
print("Conversion complete! All class names replaced with numeric IDs.")


Unknown label name: 1 in 05af7f1839.txt
Unknown label name: 1 in 05af7f1839.txt
Unknown label name: 1 in 05af7f1839.txt
Unknown label name: 1 in 05af7f1839.txt
Unknown label name: 3 in 03c77ada4d.txt
Unknown label name: 2 in 006aeeea8d.txt
Unknown label name: 2 in 006aeeea8d.txt
Unknown label name: 4 in 03b22cbe13.txt
Unknown label name: 4 in 03b22cbe13.txt
Conversion complete! All class names replaced with numeric IDs.


In [1]:
import sys, os, types, importlib.util, builtins

# --- Fake the wandb module BEFORE YOLO imports it ---
fake_wandb = types.SimpleNamespace(
    init=lambda *a, **k: None,
    log=lambda *a, **k: None,
    finish=lambda *a, **k: None,
    define_metric=lambda *a, **k: None,
    config={},
    run=None,
    Table=lambda *a, **k: None
)
sys.modules["wandb"] = fake_wandb

# Also block any re-import attempts
builtins.__import__ = (lambda orig_import:
    (lambda name, *args, **kwargs: fake_wandb if name == "wandb" else orig_import(name, *args, **kwargs))
)(__import__)

print("✅ Completely disabled wandb imports system-wide.")


✅ Completely disabled wandb imports system-wide.


In [6]:
!nohup mlflow server \
  --backend-store-uri runs/mlflow \
  --default-artifact-root runs/mlflow/artifacts \
  --host 127.0.0.1 \
  --port 5000 > mlflow.log 2>&1 &


In [7]:
!curl http://127.0.0.1:5000


<!doctype html><html lang="en"><head><meta charset="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1,shrink-to-fit=no"/><link rel="shortcut icon" href="./static-files/favicon.ico"/><meta name="theme-color" content="#000000"/><link rel="manifest" href="./static-files/manifest.json" crossorigin="use-credentials"/><title>MLflow</title><script defer="defer" src="static-files/static/js/main.875a502c.js"></script><link href="static-files/static/css/main.0139b6db.css" rel="stylesheet"></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div><div id="modal"></div></body></html>

In [8]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"
os.environ["MLFLOW_EXPERIMENT_NAME"] = "Foodalyze"

In [9]:
from ultralytics import YOLO
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Foodalyze")

with mlflow.start_run():
    model = YOLO("yolov8n.pt")
    results = model.train(
        data="/content/drive/MyDrive/Foodalyze/data/IndianFoodDatasetFinalFiltered/data.yaml",
        epochs=30,
        imgsz=640,
        name="foodalyze_yolov8_colab"
    )

    mlflow.log_param("epochs", 30)
    mlflow.log_param("imgsz", 640)
    mlflow.log_artifact("/content/runs/detect/foodalyze_yolov8_colab/weights/best.pt")


2025/10/22 19:36:17 INFO mlflow.tracking.fluent: Experiment with name 'Foodalyze' does not exist. Creating a new experiment.


Ultralytics 8.3.220 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Foodalyze/data/IndianFoodDatasetFinalFiltered/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=foodalyze_yolov8_colab, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, o

2025/10/22 19:36:49 WARNING mlflow.spark: With Pyspark >= 3.2, PYSPARK_PIN_THREAD environment variable must be set to false for Spark datasource autologging to work.
2025/10/22 19:36:49 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.


MLflow: logging run_id(a28fc25bd96249939e7d7040707947e7) to http://127.0.0.1:5000
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/runs/detect/foodalyze_yolov8_colab
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/30      2.23G     0.5695      4.562      1.083         16        640: 100% ━━━━━━━━━━━━ 145/145 2.3it/s 1:02
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 1.9it/s 5.1s
                   all        290        534     0.0152      0.299     0.0272     0.0262

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/30      2.85G     0.3591      4.094     0.9571         16        640: 100% ━━━━━━━━━━━━ 145/145 2.5it/s 57.5s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━

2025/10/22 20:08:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run clumsy-wren-608 at: http://127.0.0.1:5000/#/experiments/438376629050802937/runs/a28fc25bd96249939e7d7040707947e7.
2025/10/22 20:08:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/438376629050802937.


MLflow: results logged to http://127.0.0.1:5000
MLflow: disable with 'yolo settings mlflow=False'


2025/10/22 20:08:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run exultant-goat-449 at: http://127.0.0.1:5000/#/experiments/438376629050802937/runs/2dbfc9051279457b99131b89da80f9fb.
2025/10/22 20:08:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/438376629050802937.
